In [1]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import pandas as pd
from torch_geometric.data import Data
from torch_geometric.utils import negative_sampling
edges_df = pd.read_csv('GNN/protein_interactions.csv')

col_name = ['protein']
for i in range(1,769):
  col_name.append('feature'+str(i))
features_df = pd.read_csv('GNN/gene_embedding_GeneLLM_2.csv', header=None, names=col_name)

print(features_df)
labels_df = pd.read_csv('GNN/solubility.csv')

/tmp/ipykernel_94323/1133472201.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/tmp/ipykernel_94323/1133472201.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  edges_df = pd.read_csv('GNN/protein_interactions.csv')


      protein  feature1  feature2  feature3  feature4  feature5  feature6  \
0         FES  0.339602 -0.030744 -0.901381  0.100888  0.886443  0.383596   
1      HADHA  -0.131799 -0.025745 -0.677301 -0.053545  0.971046  0.180315   
2      SLC7A7  0.385693 -0.070692 -0.847796 -0.022054  0.959772  0.085487   
3        LCK   0.650428  0.014479 -0.866163  0.053508  0.951529  0.269402   
4       HSPA2  0.322262  0.017484 -0.849302  0.046401  0.920429  0.463832   
...       ...       ...       ...       ...       ...       ...       ...   
14445   BPY2C -0.840158 -0.042814 -0.853394 -0.049438  0.943925  0.104337   
14446    CLPS -0.270716 -0.036871 -0.915350 -0.013635  0.972046  0.016017   
14447    DNER  0.228932 -0.033579 -0.907262  0.010446  0.961684  0.524211   
14448    SOX7  0.140491  0.033339 -0.806014 -0.072016  0.938781  0.339959   
14449  CXCL14 -0.570266 -0.011502 -0.741149 -0.096209  0.967244  0.426519   

       feature7  feature8  feature9  ...  feature759  feature760  feature76

In [2]:
print(labels_df)
edges_df['combined_score'][7] = 594
edges_df_cleaned = edges_df.dropna()
print(edges_df_cleaned)
edges_df_cleaned = edges_df.dropna()
print(edges_df_cleaned)

     Gene name Solubility  Label  Word_Count Count_Category
0        ERAP2   Membrane      0         117     High Count
1     ADAMTSL5    Soluble      1          28      Low Count
2      TBC1D30   Membrane      0          55     High Count
3       KCNK18   Membrane      0         184     High Count
4         NDNF    Soluble      1         129     High Count
...        ...        ...    ...         ...            ...
1374   TRABD2B   Membrane      0          96     High Count
1375      RPS9    Soluble      1         205     High Count
1376  SLC22A16   Membrane      0          93     High Count
1377      FBN3    Soluble      1          90     High Count
1378      BDH2    Soluble      1         102     High Count

[1379 rows x 5 columns]


/tmp/ipykernel_94323/3707133632.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  edges_df['combined_score'][7] = 594


         protein1  protein2 combined_score
0            ARF5   RALGPS2            173
1            ARF5     FHDC1            154
2            ARF5  ATP6V1E1            151
3            ARF5     CYTH2            471
4            ARF5      PSD3            201
...           ...       ...            ...
13715123     LDB1    SAMD14            260
13715124     LDB1     KDM6B            161
13715125     LDB1      WWP2            229
13715126     LDB1    VPS33B            152
13715127     LDB1     NDST2            440

[13067419 rows x 3 columns]
         protein1  protein2 combined_score
0            ARF5   RALGPS2            173
1            ARF5     FHDC1            154
2            ARF5  ATP6V1E1            151
3            ARF5     CYTH2            471
4            ARF5      PSD3            201
...           ...       ...            ...
13715123     LDB1    SAMD14            260
13715124     LDB1     KDM6B            161
13715125     LDB1      WWP2            229
13715126     LDB1    VPS3

In [3]:
node_id_f = [node_id for node_id in features_df['protein']]
node_id_e1 = [node_id for node_id in edges_df_cleaned['protein1']]
node_id_e2 = [node_id for node_id in edges_df_cleaned['protein2']]
node_id_e = list(set(node_id_e1 + node_id_e2))
print(len(node_id_f))
print(len(node_id_e))

14450
18838


In [4]:
labels_df.rename(columns={'Gene name': 'protein'}, inplace=True)
labels_df

,protein,Solubility,Label,Word_Count,Count_Category
0,ERAP2,Membrane,0,117,High Count
1,ADAMTSL5,Soluble,1,28,Low Count
2,TBC1D30,Membrane,0,55,High Count
3,KCNK18,Membrane,0,184,High Count
4,NDNF,Soluble,1,129,High Count
...,...,...,...,...,...
1374,TRABD2B,Membrane,0,96,High Count
1375,RPS9,Soluble,1,205,High Count
1376,SLC22A16,Membrane,0,93,High Count
1377,FBN3,Soluble,1,90,High Count


In [5]:
nodes_in_features = set(features_df['protein'])
filtered_edges_df = edges_df_cleaned[
    edges_df_cleaned['protein1'].isin(nodes_in_features) & edges_df_cleaned['protein2'].isin(nodes_in_features)
]
# 过滤特征数据集，保留共同节点的特征
filtered_features_df = features_df

In [6]:
# 将第二列中的两种字符映射为0和1
unique_values = labels_df['Count_Category'].unique()
print(unique_values)
mapping = {unique_values[0]: 0, unique_values[1]: 1}

labels_df['Count_Category'] = labels_df['Count_Category'].map(mapping)
# 显示更新后的DataFrame
print(labels_df)

['High Count' 'Low Count']
       protein Solubility  Label  Word_Count  Count_Category
0        ERAP2   Membrane      0         117               0
1     ADAMTSL5    Soluble      1          28               1
2      TBC1D30   Membrane      0          55               0
3       KCNK18   Membrane      0         184               0
4         NDNF    Soluble      1         129               0
...        ...        ...    ...         ...             ...
1374   TRABD2B   Membrane      0          96               0
1375      RPS9    Soluble      1         205               0
1376  SLC22A16   Membrane      0          93               0
1377      FBN3    Soluble      1          90               0
1378      BDH2    Soluble      1         102               0

[1379 rows x 5 columns]


In [7]:
node_id_to_index = {node_id: i for i, node_id in enumerate(filtered_features_df['protein'])}
# 确保edge_index是按照这个新的索引顺序排列的
source_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['protein1']]
target_indices = [node_id_to_index[node_id] for node_id in filtered_edges_df['protein2']]
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)
filtered_edges_df['combined_score'] = pd.to_numeric(filtered_edges_df['combined_score'], errors='coerce', downcast='float')
edge_weight = torch.tensor(filtered_edges_df['combined_score'].values, dtype=torch.float)

# 重排特征矩阵以匹配edge_index的顺序
features = filtered_features_df.iloc[:, 1:].values
#features = features[[node_id_to_index[node_id] for node_id in filtered_features_df['protein']], :]

# 转换为PyTorch张量
features_tensor = torch.tensor(features, dtype=torch.float)

/tmp/ipykernel_94323/744925070.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges_df['combined_score'] = pd.to_numeric(filtered_edges_df['combined_score'], errors='coerce', downcast='float')


In [8]:
print(len(labels_df))
nodes_in_labels = set(labels_df['protein'])
nodes_in_filter_features = set(filtered_features_df['protein'])
common_nodes_labels = nodes_in_labels.intersection(nodes_in_filter_features)

# 过滤
filtered_labels_df = labels_df[labels_df['protein'].isin(common_nodes_labels)].reset_index(drop=True)
print(len(filtered_labels_df))
print(len(features_tensor))

1379
1379
14450


In [9]:
label_indices = [node_id_to_index[node_id] for node_id in filtered_labels_df['protein']]
print(label_indices)
num_nodes = len(features_tensor)
labels = torch.full((num_nodes,), -1, dtype=torch.long)
for i, index in enumerate(filtered_labels_df['Label']):
    labels[label_indices[i]] = index

labels_tensor = torch.tensor(labels, dtype=torch.long)

[67, 81, 93, 94, 100, 116, 129, 142, 148, 149, 157, 167, 170, 202, 228, 236, 241, 243, 255, 258, 276, 281, 287, 294, 305, 319, 321, 334, 340, 369, 373, 375, 383, 384, 399, 402, 418, 447, 455, 474, 483, 484, 490, 498, 511, 518, 526, 558, 569, 585, 586, 587, 593, 594, 595, 596, 599, 615, 641, 653, 654, 664, 682, 736, 754, 762, 764, 773, 777, 781, 833, 852, 894, 895, 899, 915, 917, 938, 944, 957, 982, 983, 997, 1003, 1019, 1040, 1047, 1053, 1062, 1064, 1065, 1067, 1074, 1075, 1078, 1079, 1105, 1108, 1115, 1116, 1120, 1124, 1131, 1136, 1137, 1143, 1157, 1170, 1173, 1183, 1195, 1196, 1197, 1215, 1235, 1236, 1237, 1243, 1256, 1261, 1262, 1273, 1274, 1277, 1311, 1315, 1327, 1333, 1347, 1363, 1371, 1377, 1378, 1383, 1392, 1397, 1426, 1427, 1431, 1435, 1444, 1445, 1469, 1470, 1475, 1484, 1489, 1492, 1493, 1521, 1527, 1540, 1548, 1559, 1575, 1593, 1606, 1609, 1614, 1617, 1619, 1637, 1638, 1645, 1674, 1681, 1710, 1720, 1733, 1736, 1737, 1759, 1776, 1795, 1807, 1823, 1833, 1834, 1844, 1848, 1849, 

/tmp/ipykernel_94323/1340105613.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(labels, dtype=torch.long)


In [10]:
highinfo_indices = [
    node_id_to_index[node_id]
    for node_id, weight in zip(filtered_labels_df['protein'], filtered_labels_df['Count_Category'])
    if weight == 0
]
print(highinfo_indices)
lowinfo_indices = [
    node_id_to_index[node_id]
    for node_id, weight in zip(filtered_labels_df['protein'], filtered_labels_df['Count_Category'])
    if weight == 1
]
print(lowinfo_indices)


[67, 93, 94, 100, 116, 129, 142, 148, 157, 167, 228, 236, 241, 255, 276, 287, 294, 321, 334, 340, 369, 373, 375, 383, 384, 399, 402, 418, 447, 474, 483, 490, 498, 511, 518, 526, 558, 569, 586, 587, 593, 594, 595, 596, 599, 615, 641, 654, 664, 682, 736, 754, 764, 777, 833, 852, 894, 895, 899, 915, 917, 938, 944, 957, 982, 983, 997, 1003, 1019, 1040, 1053, 1062, 1064, 1065, 1067, 1074, 1075, 1078, 1079, 1105, 1108, 1115, 1116, 1120, 1124, 1131, 1136, 1137, 1143, 1157, 1170, 1173, 1183, 1195, 1196, 1197, 1215, 1235, 1236, 1237, 1243, 1256, 1261, 1273, 1274, 1277, 1311, 1315, 1327, 1333, 1347, 1363, 1377, 1378, 1392, 1397, 1427, 1444, 1445, 1469, 1470, 1475, 1484, 1489, 1492, 1493, 1521, 1548, 1559, 1575, 1593, 1609, 1614, 1617, 1619, 1637, 1638, 1674, 1681, 1720, 1733, 1736, 1737, 1759, 1776, 1795, 1807, 1833, 1834, 1844, 1849, 1852, 1854, 1882, 1893, 1895, 1898, 1914, 1917, 1923, 1960, 1970, 1972, 1978, 1986, 1995, 2015, 2031, 2046, 2048, 2071, 2096, 2097, 2101, 2109, 2115, 2117, 2140, 2

In [11]:
print(len(highinfo_indices))
print(len(lowinfo_indices))

1203
176


In [12]:
data = Data(x=features_tensor, edge_index=edge_index, y=labels_tensor, edge_attr=edge_weight)

print("x:", data.x.shape, data.x.dtype)
print("edge_index:", data.edge_index.shape, data.edge_index.dtype)
print("labels:", data.y.shape, data.y.dtype)
print("edge_weight:", data.edge_attr.shape, data.edge_attr.dtype)

x: torch.Size([14450, 768]) torch.float32
edge_index: torch.Size([2, 9503503]) torch.int64
labels: torch.Size([14450]) torch.int64
edge_weight: torch.Size([9503503]) torch.float32


In [45]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class GCN(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_features, 128)
        self.conv2 = GCNConv(128, hidden_dim)
        self.fc1 = torch.nn.Linear(hidden_dim, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, num_classes)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, edge_index, edge_weight):
        # 保存初始特征
        initial_features = x

        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)

        # 在这里将初始特征和 GCN 的输出拼接在一起
        #x = torch.cat([x, initial_features], dim=1)

        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)   #应用 Sigmoid 激活函数进行逻辑回归
    def get_embeddings(self, x, edge_index, edge_weight):
        initial_features = x

        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return x

In [15]:
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

device = torch.device('cuda:1')
data = data.to(device)
# 加载数据
features = data.x
labels = data.y # 根据你的数据加载函数进行调整

original_features = features.clone()

# 初始化GNN模型
model = GCN(num_features=features.shape[1], hidden_dim=64, num_classes=1).to(device)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)

label_indices = torch.tensor(label_indices, dtype=torch.long)
print(label_indices)
# 随机打乱有标签的节点索引
labeled_indices = label_indices[torch.randperm(label_indices.size(0))]
#print(labeled_indices)
labeled_indices = label_indices

# 定义训练和测试集的大小
num_labeled = labeled_indices.size(0)
num_train = int(num_labeled * 0.8)
num_test = num_labeled - num_train
print(num_test)

# 创建训练和测试掩码
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[labeled_indices[:num_train]] = True
test_mask[labeled_indices[num_train:num_train+num_test]] = True
print(test_mask)

def train_model(model, masked_features, labels, edge_index, edge_attr, optimizer, criterion, train_mask):
    model.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index, edge_attr)  # 获取模型输出
    loss = criterion(out[train_mask], labels[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    return loss.item()

def train_model_scheduler(model, masked_features, labels, edge_index, edge_attr, optimizer, criterion, scheduler, train_mask):
    model.train()  # 设置模型为训练模
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index, edge_attr).squeeze()  # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask].float())  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    scheduler.step(loss)
    return loss.item()

def evaluate_model(model, features, labels, edge_index, edge_attr, mask):
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 关闭梯度计算
        # 获取模型输出，这里假设输出已经是经过sigmoid的概率
        probabilities = model(features, edge_index, edge_attr)[mask].squeeze()

        # 将概率转换为类别预测，这里使用0.5作为阈值
        predicted = (probabilities > 0.5).long()

        # 计算准确率
        correct = (predicted == labels[mask]).sum().item()
        acc = correct / mask.sum().item()

        # 计算AUC得分
        # 注意：这里直接使用probabilities，因为AUC需要概率值
        auc_score = roc_auc_score(labels[mask].cpu().numpy(), probabilities.cpu().numpy())
        f1 = f1_score(labels[mask].cpu().numpy(), predicted.cpu().numpy())
        return acc, auc_score, f1

 
# 总节点数
num_nodes = features.size(0)
# 所有节点的索引
all_indices = np.arange(num_nodes)

# 循环中不断更新的未被选中节点列表
remaining_indices = all_indices

num_epochs = 2500
for epoch in range(num_epochs):
    train_loss = train_model_scheduler(model, data.x, data.y, data.edge_index, data.edge_attr, optimizer, criterion, scheduler, train_mask)
    #train_loss = train_model(model, data.x, data.y, data.edge_index, data.edge_attr, optimizer, criterion, train_mask)
    test_acc, test_auc, test_f1 = evaluate_model(model, data.x, data.y, data.edge_index, data.edge_attr, test_mask)
    
    if epoch % 50 == 0:  # 每10个epoch打印一次信息
        print(f'Epoch {epoch}: Train Loss:{train_loss:.4f}, F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')

tensor([   67,    81,    93,  ..., 14398, 14407, 14426])
276
tensor([False, False, False,  ..., False, False, False])
Epoch 0: Train Loss:0.6932, F1: 0.5954, AUC_score: 0.5448
Epoch 50: Train Loss:0.6437, F1: 0.5878, AUC_score: 0.6567
Epoch 100: Train Loss:0.6141, F1: 0.5455, AUC_score: 0.6513
Epoch 150: Train Loss:0.5999, F1: 0.5596, AUC_score: 0.6560
Epoch 200: Train Loss:0.7162, F1: 0.0331, AUC_score: 0.5668
Epoch 250: Train Loss:0.5857, F1: 0.6350, AUC_score: 0.6898
Epoch 300: Train Loss:0.5645, F1: 0.5919, AUC_score: 0.6974
Epoch 350: Train Loss:0.5737, F1: 0.6717, AUC_score: 0.7347
Epoch 400: Train Loss:0.5589, F1: 0.5584, AUC_score: 0.7355
Epoch 450: Train Loss:0.5388, F1: 0.5729, AUC_score: 0.7553
Epoch 500: Train Loss:0.5659, F1: 0.5871, AUC_score: 0.7807
Epoch 550: Train Loss:0.5185, F1: 0.6132, AUC_score: 0.8102
Epoch 600: Train Loss:0.4999, F1: 0.7236, AUC_score: 0.8293
Epoch 650: Train Loss:0.5101, F1: 0.6509, AUC_score: 0.8359
Epoch 700: Train Loss:0.5008, F1: 0.7452, AUC

In [14]:
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

device = torch.device('cpu')
data = data.to(device)
# 加载数据
features = data.x
labels = data.y # 根据你的数据加载函数进行调整

original_features = features.clone()

# 初始化GNN模型
model = GCN(num_features=features.shape[1], hidden_dim=64, num_classes=1).to(device)
criterion = torch.nn.BCELoss()
#criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=100, verbose=True)

label_indices = torch.tensor(label_indices, dtype=torch.long)
print(label_indices)
# 随机打乱有标签的节点索引
labeled_indices = label_indices[torch.randperm(label_indices.size(0))]
#print(labeled_indices)
labeled_indices = label_indices

# 定义训练和测试集的大小
num_labeled = labeled_indices.size(0)
num_train = int(num_labeled * 0.8)
num_test = num_labeled - num_train
print(num_test)

# 创建训练和测试掩码
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)

train_mask[labeled_indices[:num_train]] = True
test_mask[labeled_indices[num_train:num_train+num_test]] = True
print(test_mask)

def train_model(model, masked_features, labels, edge_index, edge_attr, optimizer, criterion, train_mask):
    model.train()  # 设置模型为训练模式
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index, edge_attr)  # 获取模型输出
    loss = criterion(out[train_mask], labels[train_mask])  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    return loss.item()

def train_model_scheduler(model, masked_features, labels, edge_index, edge_attr, optimizer, criterion, scheduler, train_mask):
    model.train()  # 设置模型为训练模
    optimizer.zero_grad()  # 清空梯度
    out = model(masked_features, edge_index, edge_attr).squeeze()  # 获取模型输出
    loss = criterion(out[train_mask], data.y[train_mask].float())  # 计算损失值，只针对训练集的节点
    loss.backward()  # 反向传播
    optimizer.step()  # 更新模型参数
    scheduler.step(loss)
    return loss.item()

def evaluate_model(model, features, labels, edge_index, edge_attr, mask):
    model.eval()  # 设置模型为评估模式
    with torch.no_grad():  # 关闭梯度计算
        # 获取模型输出，这里假设输出已经是经过sigmoid的概率
        probabilities = model(features, edge_index, edge_attr)[mask].squeeze()

        # 将概率转换为类别预测，这里使用0.5作为阈值
        predicted = (probabilities > 0.5).long()

        # 计算准确率
        correct = (predicted == labels[mask]).sum().item()
        acc = correct / mask.sum().item()

        # 计算AUC得分
        # 注意：这里直接使用probabilities，因为AUC需要概率值
        auc_score = roc_auc_score(labels[mask].cpu().numpy(), probabilities.cpu().numpy())
        f1 = f1_score(labels[mask].cpu().numpy(), predicted.cpu().numpy())
        return acc, auc_score, f1


tensor([   67,    81,    93,  ..., 14398, 14407, 14426])
276
tensor([False, False, False,  ..., False, False, False])


In [46]:
model = GCN(num_features=features.shape[1], hidden_dim=64, num_classes=1).to(device)
model.load_state_dict(torch.load('GCN.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [47]:
test_acc, test_auc, test_f1 = evaluate_model(model, data.x, data.y, data.edge_index, data.edge_attr, test_mask)
print(f'F1: {test_f1:.4f}, AUC_score: {test_auc:.4f}')

TypeError: 'NoneType' object is not subscriptable

In [48]:
output = model.get_embeddings(data.x, data.edge_index, data.edge_attr)
output

tensor([[0.2713, 0.2779, 0.2735,  ..., 0.0000, 0.0000, 0.0379],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.2435],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0256],
        ...,
        [0.1471, 0.1547, 0.1509,  ..., 0.0000, 0.0000, 0.0664],
        [0.2502, 0.2556, 0.2555,  ..., 0.0000, 0.0000, 0.0348],
        [0.3536, 0.3572, 0.3565,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)

In [49]:
len(predicted)

14450

In [50]:
output.detach().numpy()

array([[0.27130732, 0.27793735, 0.27353567, ..., 0.        , 0.        ,
        0.03792006],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.2435288 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.02556284],
       ...,
       [0.14714986, 0.15466674, 0.15090339, ..., 0.        , 0.        ,
        0.06636678],
       [0.2501974 , 0.25563705, 0.2554849 , ..., 0.        , 0.        ,
        0.03484946],
       [0.3535686 , 0.35717827, 0.35645118, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [51]:
embeddings_df = pd.DataFrame(output.detach().numpy(), columns=[f'Feature_{i}' for i in range(output.shape[1])])
embeddings_df.insert(0, 'name', filtered_features_df['protein'])

In [55]:
embeddings_df.to_csv('PPI.csv', index=False)

In [52]:
labels_df

,name,Solubility,Label,Word_Count,Count_Category
0,ERAP2,Membrane,0,117,0
1,ADAMTSL5,Soluble,1,28,1
2,TBC1D30,Membrane,0,55,0
3,KCNK18,Membrane,0,184,0
4,NDNF,Soluble,1,129,0
...,...,...,...,...,...
1374,TRABD2B,Membrane,0,96,0
1375,RPS9,Soluble,1,205,0
1376,SLC22A16,Membrane,0,93,0
1377,FBN3,Soluble,1,90,0


In [53]:
labels_df.rename(columns={'protein':'name'}, inplace=True)
label_df = labels_df[['name', 'Label']]
merged_df = pd.merge(embeddings_df, label_df, on='name', how='inner')
merged_df

,name,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_55,Feature_56,Feature_57,Feature_58,Feature_59,Feature_60,Feature_61,Feature_62,Feature_63,Label
0,ERAP2,0.284831,0.287351,0.289329,0.253635,0.225623,0.184859,0.289319,0.189595,0.0,...,0.253243,0.0,0.0,0.299055,0.221455,0.305924,0.0,0.0,0.004109,0
1,ADAMTSL5,0.361459,0.352136,0.358208,0.309579,0.240752,0.204012,0.338186,0.204649,0.0,...,0.319501,0.0,0.0,0.375555,0.271339,0.375699,0.0,0.0,0.000000,1
2,TBC1D30,0.381606,0.388147,0.379526,0.335721,0.131052,0.104529,0.148468,0.110500,0.0,...,0.332554,0.0,0.0,0.390586,0.293074,0.401963,0.0,0.0,0.018417,0
3,KCNK18,0.000000,0.000000,0.000000,0.000000,0.759943,0.632143,1.044012,0.629058,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0
4,NDNF,0.249400,0.244775,0.252809,0.218678,0.323325,0.269871,0.434782,0.270781,0.0,...,0.222504,0.0,0.0,0.260071,0.191385,0.266203,0.0,0.0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,TRABD2B,0.305565,0.293708,0.302368,0.260341,0.315700,0.267527,0.444895,0.266764,0.0,...,0.270635,0.0,0.0,0.316970,0.228464,0.316689,0.0,0.0,0.000000,0
1375,RPS9,0.311494,0.330044,0.307260,0.288819,0.000000,0.000000,0.000000,0.000000,0.0,...,0.265688,0.0,0.0,0.330282,0.248358,0.330735,0.0,0.0,0.218973,1
1376,SLC22A16,0.000000,0.000000,0.000000,0.000000,0.892930,0.740275,1.228003,0.736318,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0
1377,FBN3,0.432874,0.438728,0.431085,0.380828,0.054895,0.041863,0.053413,0.048202,0.0,...,0.379332,0.0,0.0,0.449401,0.332195,0.455614,0.0,0.0,0.008581,1


In [54]:
X = merged_df.drop(columns=['name', 'Label'])
y = merged_df['Label']

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, roc_auc_score

# 标准化 embeddings
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 降维（可选）
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# 拆分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# 训练逻辑回归模型
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# 进行预测
y_pred_logistic = logistic_model.predict(X_test)
y_pred_prob_logistic = logistic_model.predict_proba(X_test)[:, 1]

# 计算逻辑回归的准确率和AUC
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
auc_logistic = roc_auc_score(y_test, y_pred_prob_logistic)
print(f'Logistic Regression Accuracy: {accuracy_logistic}')
print(f'Logistic Regression AUC: {auc_logistic}')

# 训练线性回归模型
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# 进行预测
y_pred_prob_linear = linear_model.predict(X_test)

# 将预测的连续值转换为二分类结果
y_pred_linear = (y_pred_prob_linear >= 0.5).astype(int)

# 计算线性回归的准确率和AUC
accuracy_linear = accuracy_score(y_test, y_pred_linear)
auc_linear = roc_auc_score(y_test, y_pred_prob_linear)
print(f'Linear Regression Accuracy: {accuracy_linear}')
print(f'Linear Regression AUC: {auc_linear}')

Logistic Regression Accuracy: 0.8115942028985508
Logistic Regression AUC: 0.8802787162162163
Linear Regression Accuracy: 0.782608695652174
Linear Regression AUC: 0.8815983952702702
